In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_columns', 140)
pd.set_option('max_colwidth', 5000)
pd.set_option('display.max_rows', 140)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/costa-rican-household-poverty-prediction/train.csv.zip
/kaggle/input/costa-rican-household-poverty-prediction/test.csv
/kaggle/input/costa-rican-household-poverty-prediction/sample_submission.csv
/kaggle/input/costa-rican-household-poverty-prediction/train.csv
/kaggle/input/costa-rican-household-poverty-prediction/codebook.xlsx
/kaggle/input/costa-rican-household-poverty-prediction/codebook.csv
/kaggle/input/costa-rican-household-poverty-prediction/test.csv.zip
/kaggle/input/costa-rican-household-poverty-prediction/sample_submission.csv.zip


In [88]:
## carregando e juntando datasets
train = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')
codebook = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/codebook.csv')

df = train.append(test)

df.shape


(33413, 143)

# > > Explorando dados

In [89]:
## procurando valores com grande quantidade de nulos
df.isna().sum().sort_values(ascending=False).head(10)

rez_esc           27581
v18q1             25468
v2a1              24263
Target            23856
meaneduc             36
SQBmeaned            36
epared3               0
sanitario2            0
techozinc             0
techoentrepiso        0
dtype: int64

In [90]:
## verificando variáveis que apresentaram muitos nulos e não são a Target
print(codebook[codebook['Variable name']=='rez_esc'])
print('----------------------------------')
print(codebook[codebook['Variable name']=='v18q1'])
print('----------------------------------')
print(codebook[codebook['Variable name']=='v2a1'])
print('----------------------------------')
print(codebook[codebook['Variable name']=='meaneduc'])
print('----------------------------------')
print(codebook[codebook['Variable name']=='SQBmeaned'])

   Variable name    Variable description
20       rez_esc  Years behind in school
----------------------------------
  Variable name              Variable description
7         v18q1  number of tablets household owns
----------------------------------
  Variable name  Variable description
0          v2a1  Monthly rent payment
----------------------------------
    Variable name                         Variable description
102      meaneduc  average years of education for adults (18+)
----------------------------------
    Variable name Variable description
139     SQBmeaned       meaned squared


In [91]:
df.v18q1.value_counts()

1.0    5883
2.0    1449
3.0     447
4.0     125
5.0      31
6.0      10
Name: v18q1, dtype: int64

In [92]:
df.rez_esc.value_counts()

0.0     4474
1.0      728
2.0      336
3.0      174
4.0       80
5.0       39
99.0       1
Name: rez_esc, dtype: int64

In [93]:
df.v2a1.value_counts()

100000.0    581
150000.0    579
200000.0    511
120000.0    379
50000.0     364
           ... 
268000.0      1
56000.0       1
478000.0      1
191000.0      1
234000.0      1
Name: v2a1, Length: 302, dtype: int64

In [94]:
df.meaneduc.value_counts()

6.0     3025
11.0    1514
7.0     1454
9.0     1450
8.5     1192
        ... 
33.0       3
36.0       3
23.5       3
37.0       3
32.0       2
Name: meaneduc, Length: 211, dtype: int64

In [95]:
df.SQBmeaned.value_counts()

36.00      3025
121.00     1514
49.00      1454
81.00      1450
72.25      1192
           ... 
1296.00       3
1369.00       3
1089.00       3
552.25        3
1024.00       2
Name: SQBmeaned, Length: 211, dtype: int64

* A variável v18q1 (number of tablets household owns) não possui em seus valores distintos o 0, o que leva a crer que os valores missings presentes são de casas que não possuem tabletes - Candidata a imputação de dados
* A variável rez_esc  (Years behind in school) Possui 0 em seus valores distintos, o que leva a crer que são realmente dados faltantes. - Por representar mais de 50% da base de valores nulos, é candidata a ser retirada
* A variável v2a1  (Monthly rent payment) só é preenchida em caso de casas alugadas, para isso utilizaremos o número -1 nos valores nulos
* A variável meaneduc (average years of education for adults (18+)) é candidata a ter seus valores nulos substituidos pela mediana
* A variável SQBmeaned (meaned squared) é a raiz quadrada da variável meaneduc, por ser derivada de outra coluna, é candidata a ser retirada

In [96]:
#realizando a imputação e dropando coluna
df['v2a1'].fillna(-1, inplace=True)
df['v18q1'].fillna(0, inplace=True)
df.drop(['rez_esc'], axis=1, inplace=True)
df.drop(['SQBmeaned'], axis=1, inplace=True)
df.meaneduc.fillna(df.meaneduc.median(), inplace=True)
## realizando o drop de outras colunas que assim como SQBmeaned são provenientes de outras
df.drop(['SQBescolari'], axis=1, inplace=True)
df.drop(['SQBage'], axis=1, inplace=True)
df.drop(['SQBhogar_total'], axis=1, inplace=True)
df.drop(['SQBedjefe'], axis=1, inplace=True)
df.drop(['SQBhogar_nin'], axis=1, inplace=True)
df.drop(['SQBovercrowding'], axis=1, inplace=True)
df.drop(['SQBdependency'], axis=1, inplace=True)
df.drop(['agesq'], axis=1, inplace=True)

In [97]:
df.isna().sum().sort_values(ascending=False).head(10)

Target             23856
sanitario1             0
pisonotiene            0
pisomadera             0
techozinc              0
techoentrepiso         0
techocane              0
techootro              0
cielorazo              0
abastaguadentro        0
dtype: int64

In [105]:
##verificando se existem colunas do tipo objeto
df.select_dtypes('object').describe()

,Id,idhogar,dependency,edjefe,edjefa
count,33413,33413,33413,33413,33413
unique,33413,10340,38,22,22
top,ID_74eb4c642,9a906088e,yes,no,no
freq,1,13,7580,12818,22075


In [107]:
df.dependency.value_counts().head(5)

yes    7580
no     6036
.5     5175
2      2499
1.5    2471
Name: dependency, dtype: int64

In [108]:
df.edjefe.value_counts().head(5)

no    12818
6      6958
11     2507
9      1564
3      1046
Name: edjefe, dtype: int64

In [109]:
df.edjefa.value_counts().head(5)

no    22075
6      3401
11     1350
9       735
15      656
Name: edjefa, dtype: int64

In [110]:
## dropando Id e idhogar e realizando um replace nas restantes
valores_replace = {'yes': 1, 'no': 0}
df.drop(['Id'], axis=1, inplace=True)
df.drop(['idhogar'], axis=1, inplace=True)
df['dependency'] = df['dependency'].replace(valores_replace).astype(float)
df['edjefe'] = df['edjefe'].replace(valores_replace).astype(int)
df['edjefa'] = df['edjefa'].replace(valores_replace).astype(int)

In [ ]:
df.select_dtypes('object')